
# Output oriented model

The following DEA model is an output-oriented model where the outputs are maximizing and the inputs are kept at their current levels.

\begin{align}& \theta^* = \min \theta, subject \ to \\
    & \sum_{j=1}^{n} \lambda_j x_{i, j} \leq x_{i, o}, i=1,2, \dots, m; \\
    & \sum_{j=1}^{n} \lambda_j y_{r, j} \geq \theta y_{r, o}, r=1,2, \dots, s; \\
    & \sum_{j=1}^{n} \lambda_j = 1 \\
    & \lambda_j \geq 0, j=1,2, \dots, n.\end{align}

where $DMU_o$ represents one of the $n$ DMUs under evaluation,
and $x_{i, o}$ and $y_{i, o}$ are the $i$ th input and $r$ th output
for $DMU_o$, respectively.


## Import modules and prepare data.
Average of rental properties in a given district.



In [4]:
import matplotlib.pyplot as plt
import pandas as pd

from Pyfrontier.frontier_model import EnvelopDEA

rent_average_df = pd.DataFrame(
    {"rent": [5, 5, 5, 5], "n_room": [6, 5, 2, 3.0], "n_storage": [2, 3.5, 5, 3.5]}
)
rent_average_df

,rent,n_room,n_storage
0,5,6.0,2.0
1,5,5.0,3.5
2,5,2.0,5.0
3,5,3.0,3.5


## Fit dea model.

The necessity inputs are inputs and outputs. The result has below belongings.



In [5]:
import pulp as pl
print(pl.pulp_cbc_path)
print("Available solvers:", pl.listSolvers(onlyAvailable=True))

/Users/moky/miniconda3/envs/johi_env/lib/python3.13/site-packages/pulp/solverdir/cbc/osx/arm64/cbc
Available solvers: ['GLPK_CMD', 'COIN_CMD']


In [6]:
dea = EnvelopDEA("CRS", "out")
dea.fit(
    rent_average_df[["rent"]].to_numpy(),
    rent_average_df[["n_room", "n_storage"]].to_numpy(),
    
)

dea.result[0]

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /var/folders/_y/krk2nfs579bdmxnfc1j2_cpc0000gn/T/989ad7a3e4f14a46838b9a42ad33be6e-pulp.lp
 -o /var/folders/_y/krk2nfs579bdmxnfc1j2_cpc0000gn/T/989ad7a3e4f14a46838b9a42ad33be6e-pulp.sol
Reading problem data from '/var/folders/_y/krk2nfs579bdmxnfc1j2_cpc0000gn/T/989ad7a3e4f14a46838b9a42ad33be6e-pulp.lp'...
3 rows, 5 columns, 14 non-zeros
8 lines were read
GLPK Simplex Optimizer 5.0
3 rows, 5 columns, 14 non-zeros
Preprocessing...
3 rows, 5 columns, 14 non-zeros
Scaling...
 A: min|aij| =  2.000e+00  max|aij| =  6.000e+00  ratio =  3.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (1)
*     2: obj =   1.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (32525 bytes)
Writing basic solution to '/var/folders/_y/krk2nfs579bdmxnfc1j2_cpc0000gn/T/989ad7a3e

PulpSolverError: PULP_CBC_CMD: Not Available (check permissions on /Users/moky/miniconda3/envs/johi_env/lib/python3.13/site-packages/pulp/solverdir/cbc/osx/arm64/cbc)

## Visualize the result.

In the built documentation.



In [ ]:
eff_dmu = [r.dmu for r in dea.result if r.is_efficient]
ineff_dmu = [r.dmu for r in dea.result if r.is_efficient != 1]

plt.figure()
plt.plot(
    [d.output[0] for d in eff_dmu],
    [d.output[1] for d in eff_dmu],
    "-o",
    label="efficient dmu",
)
plt.plot(
    [d.output[0] for d in ineff_dmu],
    [d.output[1] for d in ineff_dmu],
    "o",
    label="not-efficient dmu",
)
plt.plot([6, 6], [2, 0], color="C0")
plt.plot([2, 0], [5, 5], color="C0")
plt.plot([0, 3.6], [0, 4.2], color="black", linestyle="--")

plt.legend()
plt.show()

## About slack

In the built documentation.



In [ ]:
print([r.score for r in dea.result])
print([r.is_efficient for r in dea.result])
print([r.has_slack for r in dea.result])

print(dea.result[-2].x_slack, dea.result[-2].y_slack)